# 1. Configuring the Nodes

Server = main node and worker nodes were configured according to the script below


In [3]:
cat ../pastebins/05L_vid_p2_installconfigure_an_NFS_serverclient.sh

#on the server:

 yum install nfs-utils rpcbind

 # Used to enable nfs starting at the boot of the machine
 systemctl enable nfs-server
 systemctl enable rpcbind     # starts the boot
 systemctl enable nfs-lock
 systemctl enable nfs-idmap 
 systemctl start rpcbind      # starting the service immediately
 systemctl start nfs-server
 systemctl start nfs-lock
 systemctl start nfs-idmap

 
 systemctl status nfs   ### Checking that everything is fine 'system control'
# you should see:
#  Loaded: loaded (/usr/lib/systemd/system/nfs-server.service; enabled; vendor preset: disabled)
#    Active: active (exited) since Thu 2021-02-18 16:33:02 UTC; 1min 46s ago
#   Process: 16053 ExecStartPost=/bin/sh -c if systemctl -q is-active gssproxy; then systemctl reload gssproxy ; fi (code=exited, status=0/SUCCESS)
#   Process: 16035 ExecStart=/usr/sbin/rpc.nfsd $RPCNFSDARGS (code=exited, status=0/SUCCESS)
#   Process: 16033 ExecStartPre=/usr/sbin/exportfs -r (code=exited, status=0/SUCCESS)
#  Main PID: 1

# 2. Install HT condor machine

In [4]:
# cat ../pastebins/06L_p1_condor_install.sh

#INSTALL DEPENDENCIES
yum install wget
wget https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
yum localinstall epel-release-latest-7.noarch.rpm
yum clean all


#INSTALL CONDOR REPOs and PACKAGES
wget http://research.cs.wisc.edu/htcondor/yum/repo.d/htcondor-stable-rhel7.repo
cp htcondor-stable-rhel7.repo /etc/yum.repos.d/
wget http://htcondor.org/yum/RPM-GPG-KEY-HTCondor
rpm --import RPM-GPG-KEY-HTCondor
yum install condor-all

#CONDOR BASIC CONFIGURATION
cd
vim /etc/condor/condor_config
systemctl status condor 
systemctl start condor 
systemctl enable condor
systemctl status condor 
ps -aux | grep condor

# END - SOME HINTS BELOW
1) Security Group must allow tcp for ports 0 - 65535 from the same security group, i.e.:
 All TCP    TCP      0 - 65535     sg-008742ba0467986fe (aws_condor)
Security group must allow ping from the same security group, i.e.:
 All    ICMP-IPv4   All    N/A     sg-008742ba0467986fe (aws_condor)
Security group must allow ssh on port 22 from e

## Setting up master:

In [ ]:
vim /etc/condor/condor_config # jump to line 36 well add the line 40
#-------------------------------------
# In the config file add at the end
# the most important variable is the CONDOR_HOST running the master

# ADD the following lines to your condor_config file
# CHANGE THE FOLLOWING IP TO YOUR MASTER IP
CONDOR_HOST = __put here the master Private IP address, i.e.: 172.31.25.191 ___ # add to Main and ALL NODES
# Condor host is the main!
# ONLY on the master
DAEMON_LIST = COLLECTOR, MASTER, NEGOTIATOR, STARTD, SCHEDD

HOSTALLOW_READ = *
HOSTALLOW_WRITE = *
HOSTALLOW_ADMINISTRATOR = *
#---------------------------------------------

### Setting up stateless worker nodes:

In [ ]:
vim /etc/condor/condor_config # jump to line 36 well add the line 40
#-------------------------------------
# In the config file add at the end
# the most important variable is the CONDOR_HOST running the master

# ADD the following lines to your condor_config file
# CHANGE THE FOLLOWING IP TO YOUR MASTER IP
CONDOR_HOST = __put here the master Private IP address, i.e.: 172.31.25.191 ___ # add to Main and ALL NODES
# Condor host is the main!

# #ONLY on the nodes
DAEMON_LIST = MASTER, STARTD

HOSTALLOW_READ = *
HOSTALLOW_WRITE = *
HOSTALLOW_ADMINISTRATOR = *
#---------------------------------------------



## Modifying /etc/exportfs

* By adding a line that is valid for all worker nodes:
* This is done by exploiting the CIDIR notation 
* All wn are in the SAME subnet!




In [ ]:
/data2 <first_5_digits_of_private_ip/16>